# Develop an Embedding + CNN Model for Sentiment Analysis

Having the similar meaning and a similar real-valued vector representation for the word embedding techinique to represent a text of different words.

# 15.1 Movie Review Dataset

The Dataset is designed for the sentimental analysis which was described http://www.cs.cornell.edu/people/pabo/movie-review-data/review_polarity.tar.gz 
Movie Review Polarity Dataset

# 15.2 Data Preparation

Preparation for the movie dataset was already described

1. Seperation of data into training and test sets.

2. Loading and Cleaning the data to remove ppuncutation and numbers

3. Defining a Vocabulary of preferred words

# 15.2.1 Split into Train and Test Sets

Predecting what we have been developed a system which can predict the sentiment of the textual movie review either it may be positive or a negative which means after developing the model we need to make the predicitons for the new text reviews,requiring all the same data preparation that could be performed on the new type reviews which was been performed on the training data for that kind of model,ensuring the constraint which was built in the evalluation of our model was by splitting the training and the test datasets are prior to any other data preparations,the split can be imposed easily by using the filenames of the reviews where those are named 000 to 899 and also for the traning data and reviews which are named 900.

# 15.2.2 Loading and Cleaning Reviews

As the text was already pretty clean and not much preparations was required without getting bogged down too much in details we are going to prepare the data using 

Split tokens on white space

Removing all the puncutations from words

Removing all words that are not purely comprised of alphabetical characters

Removing all the words that are known to stop the words

Removing all the words that are having a length less than or equal to 1 character

putting all of this steps together tooking step into a function called clean_doc() which takes the argument of the raw text loaded from a file and that returns a list of the cleaned tokens defining a function load_doc() which loads a document when a file was ready for using the clean_doc() function.

In [0]:
# from nltk.corpus import stopwords
import string
import re
# load doc into memory
def load_doc(filename):
# open the file as read only
file = open(filename, 'r')
# read all text
text = file.read()
# close the file
file.close()
return text
# turn a doc into clean tokens
def clean_doc(doc):
# split into tokens by white space
tokens = doc.split()
# prepare regex for char filtering
re_punc = re.compile('[%s]' % re.escape(string.punctuation))
# remove punctuation from each word
tokens = [re_punc.sub('', w) for w in tokens]
# remove remaining tokens that are not alphabetic
tokens = [word for word in tokens if word.isalpha()]
# filter out stop words
stop_words = set(stopwords.words('english'))
tokens = [w for w in tokens if not w in stop_words]
# filter out short tokens
tokens = [word for word in tokens if len(word) > 1]
return tokens
# load the document
filename = 'txt_sentoken/pos/cv000_29590.txt'
text = load_doc(filename)
tokens = clean_doc(text)
print(tokens)

# 15.2.3 Define a Vocabulary

As it was one of the important for defining a vocabulary of the known words for using a text model the more words the larger representations of the documents as it was important to constrain the words only those who are belived to be more predictive as it was difficult to know beforehand and often as it was important to test the different hypotheses about how to construct a useful vocabulary as we already seen how to remove the punctuation and also how to build a set of all the known words,developing a vocabulary as a counter the dictionary mapping of the words and the count which allows us to update the query as the each document which can be added to a counter that can step all over the reviews in the both ways either it may be the negative directory or the positive directory.

In [0]:
import string
import re
from os import listdir
from collections import Counter
from nltk.corpus import stopwords
# load doc into memory
def load_doc(filename):
# open the file as read only
file = open(filename, 'r')
# read all text
text = file.read()
# close the file
file.close()
return text
# turn a doc into clean tokens
def clean_doc(doc):
# split into tokens by white space
tokens = doc.split()
# prepare regex for char filtering
re_punc = re.compile('[%s]' % re.escape(string.punctuation))
# remove punctuation from each word
tokens = [re_punc.sub('', w) for w in tokens]
# remove remaining tokens that are not alphabetic
tokens = [word for word in tokens if word.isalpha()]
# filter out stop words
stop_words = set(stopwords.words('english'))
tokens = [w for w in tokens if not w in stop_words]
# filter out short tokens
tokens = [word for word in tokens if len(word) > 1]
return tokens
# load doc and add to vocab
def add_doc_to_vocab(filename, vocab):
# load doc
doc = load_doc(filename)
# clean doc
tokens = clean_doc(doc)
# update counts
vocab.update(tokens)
# load all docs in a directory
def process_docs(directory, vocab):
# walk through all files in the folder
for filename in listdir(directory):
# skip any reviews in the test set
if filename.startswith('cv9'):
continue
# create the full path of the file to open
path = directory + '/' + filename
# add doc to vocab
add_doc_to_vocab(path, vocab)
# define vocab
vocab = Counter()
# add all docs to vocab
process_docs('txt_sentoken/pos', vocab)
process_docs('txt_sentoken/neg', vocab)
# print the size of the vocab
print(len(vocab))
# print the top words in the vocab
print(vocab.most_common(50))

stepping through the vocabulary to remove all the words that was having a low occurences such as only being used once or twice in all the reviews.

In [0]:
# keep tokens with a min occurrence
min_occurane = 2
tokens = [k for k,c in vocab.items() if c >= min_occurane]
print(len(tokens))

Vocabulary can be saved in a new file called vocab.txt which is later loaded and used to filter the movie reviews prior to encoding and also for the modelling defining a new function called save_list() which saves the vocabulary to the file one word per line

In [0]:
# save list to file
def save_list(lines, filename):
# convert lines to a single blob of text
data = '\n'.join(lines)
# open file
file = open(filename, 'w')
# write text
file.write(data)
# close file
file.close()
# save tokens to a vocabulary file
save_list(tokens, 'vocab.txt')

Pulling all this together

In [0]:
import string
import re
from os import listdir
from collections import Counter
from nltk.corpus import stopwords
# load doc into memory
def load_doc(filename):
# open the file as read only
file = open(filename, 'r')
# read all text
text = file.read()
# close the file
file.close()
return text
# turn a doc into clean tokens
def clean_doc(doc):
# split into tokens by white space
tokens = doc.split()
# prepare regex for char filtering
re_punc = re.compile('[%s]' % re.escape(string.punctuation))
# remove punctuation from each word
tokens = [re_punc.sub('', w) for w in tokens]
# remove remaining tokens that are not alphabetic
tokens = [word for word in tokens if word.isalpha()]
# filter out stop words
stop_words = set(stopwords.words('english'))
tokens = [w for w in tokens if not w in stop_words]
# filter out short tokens
tokens = [word for word in tokens if len(word) > 1]
return tokens
# load doc and add to vocab
def add_doc_to_vocab(filename, vocab):
# load doc
doc = load_doc(filename)
# clean doc
tokens = clean_doc(doc)
# update counts
vocab.update(tokens)
# load all docs in a directory
def process_docs(directory, vocab):
# walk through all files in the folder
for filename in listdir(directory):
# skip any reviews in the test set
if filename.startswith('cv9'):
continue
# create the full path of the file to open
path = directory + '/' + filename
# add doc to vocab
add_doc_to_vocab(path, vocab)
# save list to file
def save_list(lines, filename):
# convert lines to a single blob of text
data = '\n'.join(lines)
# open file
file = open(filename, 'w')
# write text
file.write(data)
# close file
file.close()
# define vocab
vocab = Counter()
# add all docs to vocab
process_docs('txt_sentoken/pos', vocab)
process_docs('txt_sentoken/neg', vocab)
# print the size of the vocab
print(len(vocab))
# keep tokens with a min occurrence
min_occurane = 2
tokens = [k for k,c in vocab.items() if c >= min_occurane]
print(len(tokens))
# save tokens to a vocabulary file
save_list(tokens, 'vocab.txt')

# 15.3 Train CNN With Embedding Layer

Learning a word embedding while training a convolutional neural network for the classification as the word embedding is the only way representing the text where the each word in the vocabulary is represente by a real valued vector oon a high dimensional space and the vectors that are learned in a way that the words having the similar meanings and also the similar representations in the vector space and the relationships between the words and tokens are ignored or forced in bigram and trigram approaches.

Real valued vector represents for the words that can be learned while training the neural network which can do the kind of keras deep learning library using the embedding layer as the first step is to load the vocabulary.

In [0]:
# load doc into memory
def load_doc(filename):
# open the file as read only
file = open(filename, 'r')
# read all text
text = file.read()
# close the file
file.close()
return text
# load the vocabulary
vocab_filename = 'vocab.txt'
vocab = load_doc(vocab_filename)
vocab = set(vocab.split())

The updated clean_doc()function

In [0]:
# turn a doc into clean tokens
def clean_doc(doc, vocab):
# split into tokens by white space
tokens = doc.split()
# prepare regex for char filtering
re_punc = re.compile('[%s]' % re.escape(string.punctuation))
# remove punctuation from each word
tokens = [re_punc.sub('', w) for w in tokens]
# filter out tokens not in vocab
tokens = [w for w in tokens if w in vocab]
tokens = ' '.join(tokens)
return tokens

The updated process_docs()

In [0]:
# load all docs in a directory
def process_docs(directory, vocab, is_train):
documents = list()
# walk through all files in the folder
for filename in listdir(directory):
# skip any reviews in the test set
if is_train and filename.startswith('cv9'):
continue
if not is_train and not filename.startswith('cv9'):
continue
# create the full path of the file to open
path = directory + '/' + filename
# load the doc
doc = load_doc(path)
# clean doc
tokens = clean_doc(doc, vocab)
# add to list
documents.append(tokens)
return documents

As the process_docs function for the both negative as well as positive directories combining together into a single train by defining the class labels for the dataset.

In [0]:
# load and clean a dataset
def load_clean_dataset(vocab, is_train):
# load documents
neg = process_docs('txt_sentoken/neg', vocab, is_train)
pos = process_docs('txt_sentoken/pos', vocab, is_train)
docs = neg + pos
# prepare labels
labels = array([0 for _ in range(len(neg))] + [1 for _ in range(len(pos))])
return docs, labels

Moving on to the next step the encode of the each document as a sequence of integers as the keras embedding layer requires the integer inputs where the integer maps to a single token that having a specific beginning of traning but in traning it became meaningful to the specified network.Create_tokenizer() function below is preparing a tokenizer for the training data.

In [0]:
# fit a tokenizer
def create_tokenizer(lines):
tokenizer = Tokenizer()
tokenizer.fit_on_texts(lines)
return tokenizer

Mapping of the words to the integers that has been prepared and we can use it for encoding the reviews in the training dataset by calling the texts_to_sequences() functon on the tokenizer for that we need all documents are having the same length.Calling out the Keras function pad_sequnces() the pad of the sequences for a maximum length by adding 0 values on the end.

In [0]:
max_length = max([len(s.split()) for s in train_docs])
print('Maximum length: %d' % max_length)

Using the maximum length as a parameter to a function the integer encode the pad sequence.

In [0]:
# integer encode and pad documents
def encode_docs(tokenizer, max_length, docs):
# integer encode
encoded = tokenizer.texts_to_sequences(docs)
# pad sequences
padded = pad_sequences(encoded, maxlen=max_length, padding='post')
return padded

Ready to define the neural network model while using the model embedding layer as the first hidden layer and the embedding layer requires the specification for the vocabulary size and the size of the real-valued vector space having the maximum length of the input documents.

In [0]:
# define vocabulary size
vocab_size = len(tokenizer.word_index) + 1
print('Vocabulary size: %d' % vocab_size)

Using the 100 dimensional vector space but also trying the other values such as 50 ro 150 and the maximum document length was calculated above by max_length variable used during the padding a conservative CNN configuration is used with 32 filters (parallel fields for processing words) and a kernel size of 8 with a rectified linear (relu) activation function.This is followed by a pooling layer that reduces the output of the convolutional layer by half.

In [0]:
# define the model
def define_model(vocab_size, max_length):
model = Sequential()
model.add(Embedding(vocab_size, 100, input_length=max_length))
model.add(Conv1D(filters=32, kernel_size=8, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(10, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
# compile network
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# summarize defined model
model.summary()
plot_model(model, to_file='model.png', show_shapes=True)
return model

getting a better result with a different configuration 

In [0]:
# fit network
model.fit(Xtrain, ytrain, epochs=10, verbose=2)

After this model is fitted it was saved to a file name model.h5 for later

In [0]:
# save the model
model.save('model.h5')

Putting all this together

In [0]:
import string
import re
from os import listdir
from numpy import array
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils.vis_utils import plot_model
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Embedding
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
# load doc into memory
def load_doc(filename):
# open the file as read only
file = open(filename, 'r')
# read all text
text = file.read()
# close the file
file.close()
return text
# turn a doc into clean tokens
def clean_doc(doc, vocab):
# split into tokens by white space
tokens = doc.split()
# prepare regex for char filtering
re_punc = re.compile('[%s]' % re.escape(string.punctuation))
# remove punctuation from each word
tokens = [re_punc.sub('', w) for w in tokens]
# filter out tokens not in vocab
tokens = [w for w in tokens if w in vocab]
tokens = ' '.join(tokens)
return tokens
# load all docs in a directory
def process_docs(directory, vocab, is_train):
documents = list()
# walk through all files in the folder
for filename in listdir(directory):
# skip any reviews in the test set
if is_train and filename.startswith('cv9'):
continue
if not is_train and not filename.startswith('cv9'):
continue
# create the full path of the file to open
path = directory + '/' + filename
# load the doc
doc = load_doc(path)
# clean doc
tokens = clean_doc(doc, vocab)
# add to list
documents.append(tokens)
return documents
# load and clean a dataset
def load_clean_dataset(vocab, is_train):
# load documents
neg = process_docs('txt_sentoken/neg', vocab, is_train)
pos = process_docs('txt_sentoken/pos', vocab, is_train)
docs = neg + pos
# prepare labels
labels = array([0 for _ in range(len(neg))] + [1 for _ in range(len(pos))])
return docs, labels
# fit a tokenizer
def create_tokenizer(lines):
tokenizer = Tokenizer()
tokenizer.fit_on_texts(lines)
return tokenizer
# integer encode and pad documents
def encode_docs(tokenizer, max_length, docs):
# integer encode
encoded = tokenizer.texts_to_sequences(docs)
# pad sequences
padded = pad_sequences(encoded, maxlen=max_length, padding='post')
return padded
# define the model
def define_model(vocab_size, max_length):
model = Sequential()
model.add(Embedding(vocab_size, 100, input_length=max_length))
model.add(Conv1D(filters=32, kernel_size=8, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(10, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
# compile network
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# summarize defined model
model.summary()
plot_model(model, to_file='model.png', show_shapes=True)
return mode
# load the vocabulary
vocab_filename = 'vocab.txt'
vocab = load_doc(vocab_filename)
vocab = set(vocab.split())
# load training data
train_docs, ytrain = load_clean_dataset(vocab, True)
# create the tokenizer
tokenizer = create_tokenizer(train_docs)
# define vocabulary size
vocab_size = len(tokenizer.word_index) + 1
print('Vocabulary size: %d' % vocab_size)
# calculate the maximum sequence length
max_length = max([len(s.split()) for s in train_docs])
print('Maximum length: %d' % max_length)
# encode data
Xtrain = encode_docs(tokenizer, max_length, train_docs)
# define model
model = define_model(vocab_size, max_length)
# fit network
model.fit(Xtrain, ytrain, epochs=10, verbose=2)
# save the model
model.save('model.h5')

#  15.4 Evaluate Model

Evaluating the trained model and using it to make the predicitions on the new data by firstly built-in evaluate() function to estimating the skills of the model on both training and the test dataset as this requries the load and encode of both the training and test datasets.

In [0]:
# load all reviews
train_docs, ytrain = load_clean_dataset(vocab, True)
test_docs, ytest = load_clean_dataset(vocab, False)
# create the tokenizer
tokenizer = create_tokenizer(train_docs)
# define vocabulary size
vocab_size = len(tokenizer.word_index) + 1
print('Vocabulary size: %d' % vocab_size)
# calculate the maximum sequence length
max_length = max([len(s.split()) for s in train_docs])
print('Maximum length: %d' % max_length)
# encode data
Xtrain = encode_docs(tokenizer, max_length, train_docs)
Xtest = encode_docs(tokenizer, max_length, test_docs)

we can load the model and we can evaluate it on the both datasets.

In [0]:
# load the model
model = load_model('model.h5')
# evaluate model on training dataset
_, acc = model.evaluate(Xtrain, ytrain, verbose=0)
print('Train Accuracy: %f' % (acc*100))
# evaluate model on test dataset
_, acc = model.evaluate(Xtest, ytest, verbose=0)
print('Test Accuracy: %f' % (acc*100))

new data must be prepared using the same text encoding and encoding schemes as was used on the training dataset and the function below named predict_sentiment() will encoding the pad which was given a movie review text and return a prediction in terms of both the precentage and the label.

In [0]:
# classify a review as negative or positive
def predict_sentiment(review, vocab, tokenizer, max_length, model):
# clean review
line = clean_doc(review, vocab)
# encode and pad review
padded = encode_docs(tokenizer, max_length, [line])
# predict sentiment
yhat = model.predict(padded, verbose=0)
# retrieve predicted percentage and label
percent_pos = yhat[0,0]
if round(percent_pos) == 0:
return (1-percent_pos), 'NEGATIVE'
return percent_pos, 'POSITIVE'

Testing out the two ad hoc movie reviews, the complete code

In [0]:
import string
import re
from os import listdir
from numpy import array
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import load_model
# load doc into memory
def load_doc(filename):
# open the file as read only
file = open(filename, 'r')
# read all text
text = file.read()
# close the file
file.close()
return text
# turn a doc into clean tokens
def clean_doc(doc, vocab):
# split into tokens by white space
tokens = doc.split()
# prepare regex for char filtering
re_punc = re.compile('[%s]' % re.escape(string.punctuation))
# remove punctuation from each word
tokens = [re_punc.sub('', w) for w in tokens]
# filter out tokens not in vocab
tokens = [w for w in tokens if w in vocab]
tokens = ' '.join(tokens)
return tokens
# load all docs in a directory
def process_docs(directory, vocab, is_train):
documents = list()
# walk through all files in the folder
for filename in listdir(directory):
# skip any reviews in the test set
if is_train and filename.startswith('cv9'):
continue
if not is_train and not filename.startswith('cv9'):
continue
# create the full path of the file to open
path = directory + '/' + filename
# load the doc
doc = load_doc(path)
# clean doc
tokens = clean_doc(doc, vocab)
# add to list
documents.append(tokens)
return documents
# load and clean a dataset
def load_clean_dataset(vocab, is_train):
# load documents
neg = process_docs('txt_sentoken/neg', vocab, is_train)
pos = process_docs('txt_sentoken/pos', vocab, is_train)
docs = neg + pos
# prepare labels
labels = array([0 for _ in range(len(neg))] + [1 for _ in range(len(pos))])
return docs, labels
# fit a tokenizer
def create_tokenizer(lines):
tokenizer = Tokenizer()
tokenizer.fit_on_texts(lines)
return tokenizer
# integer encode and pad documents
def encode_docs(tokenizer, max_length, docs):
# integer encode
encoded = tokenizer.texts_to_sequences(docs)
# pad sequences
padded = pad_sequences(encoded, maxlen=max_length, padding='post')
return padded
# classify a review as negative or positive
def predict_sentiment(review, vocab, tokenizer, max_length, model):
# clean review
line = clean_doc(review, vocab)
# encode and pad review
padded = encode_docs(tokenizer, max_length, [line])
# predict sentiment
yhat = model.predict(padded, verbose=0)
# retrieve predicted percentage and label
percent_pos = yhat[0,0]
if round(percent_pos) == 0:
return (1-percent_pos), 'NEGATIVE'
return percent_pos, 'POSITIVE'
# load the vocabulary
vocab_filename = 'vocab.txt'
vocab = load_doc(vocab_filename)
vocab = set(vocab.split())
# load all reviews
train_docs, ytrain = load_clean_dataset(vocab, True)
test_docs, ytest = load_clean_dataset(vocab, False)
# create the tokenizer
tokenizer = create_tokenizer(train_docs)
# define vocabulary size
vocab_size = len(tokenizer.word_index) + 1
print('Vocabulary size: %d' % vocab_size)
# calculate the maximum sequence length
max_length = max([len(s.split()) for s in train_docs])
print('Maximum length: %d' % max_length)
# encode data
Xtrain = encode_docs(tokenizer, max_length, train_docs)
Xtest = encode_docs(tokenizer, max_length, test_docs)
# load the model
model = load_model('model.h5')
# evaluate model on training dataset
_, acc = model.evaluate(Xtrain, ytrain, verbose=0)
print('Train Accuracy: %.2f' % (acc*100))
# evaluate model on test dataset
_, acc = model.evaluate(Xtest, ytest, verbose=0)
print('Test Accuracy: %.2f' % (acc*100))
# test positive text
text = 'Everyone will enjoy this film. I love it, recommended!'
percent, sentiment = predict_sentiment(text, vocab, tokenizer, max_length, model)
print('Review: [%s]\nSentiment: %s (%.3f%%)' % (text, sentiment, percent*100))
# test negative text
text = 'This is a bad movie. Do not watch it. It sucks.'
percent, sentiment = predict_sentiment(text, vocab, tokenizer, max_length, model)
print('Review: [%s]\nSentiment: %s (%.3f%%)' % (text, sentiment, percent*100))

# 15.5 Extensions

Data Cleaning : Exploring the better data cleaning perhaps leaving some puncutations in  tact or normalizing the contractions

Truncated Sequences : Padding all the sequences upto a length for a longest sequence that might be a extreme if the longeset sequence is very different to all other reviews studying the distribution of a review lengths and the truncate reviews to a mean length.

Truncated Vocabulary: Removing the infrequently occuring words and had a large vocabulary of more than 25,000 words and exploring the further reducing the size of the vocabulary and the effect on the model skill

Filters and Kernel Size :  Number of filters and kernel sizes are more important to the model skill and those were not tuned exploring the tuning these two CNN parameters.

Epochs and Batch Size : As the model appears to be fit in the training dataset quickly exploring the alternate configurations of the number of training epochs and the batch size which are using the test dataset as a validation set for picking a better stop point for training the model.

Deeper Network : Explore whether a deeper network results in better skill, either in terms of CNN layers, MLP layers and both.

Pre-Train an Embedding : Explore pre-training a Word2Vec word embedding in the model and the impact on model skill with and without further fine tuning during training.

Use GloVe Embedding : Explore loading the pre-trained GloVe embedding and the impact on model skill with and without further ne tuning during training.

Longer Test Reviews: Explore whether the skill of model predictions is dependent on the length of movie reviews as suspected in the nal section on evaluating the model.

Train Final Model: Train a final model on all available data and use it make predictions on real ad hoc movie reviews from the internet.